In [1]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [8]:
dataset = pd.read_table('../Datasets/Diabetic Retinopathy/messidor_features.arff', sep=',', index_col=False, header=None)

In [9]:
X, y = dataset.iloc[:, :-1], dataset.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# AUTOML

In [11]:
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, y_train)
y_hat = automl.predict(X_test)

In [12]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.7569444444444444

In [13]:
import pickle
with open('./models/diabetic_retinopathy_automl.pkl', 'wb') as f:
    pickle.dump(automl, f)

In [4]:
import pickle
with open('./models/diabetic_retinopathy_automl.pkl', 'rb') as f:
    automl = pickle.load(f)

automl.show_models()

{58: {'model_id': 58,
  'rank': 1,
  'cost': 0.19649122807017538,
  'ensemble_weight': 0.08,
  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice at 0x7f2b737ab050>,
  'balancing': Balancing(random_state=1),
  'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice at 0x7f2b7377c850>,
  'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice at 0x7f2b736cc550>,
  'sklearn_classifier': MLPClassifier(activation='tanh', alpha=0.009705401196191985, beta_1=0.999,
                beta_2=0.9, hidden_layer_sizes=(112, 112),
                learning_rate_init=0.0006546373019923048, max_iter=256,
                n_iter_no_change=32, random_state=1, validation_fraction=0.0,
                verbose=0, warm_start=True)},
 69: {'model_id': 69,
  'rank': 2,
  'cost': 0.19649122807017538,
  'ensemble_weight': 0.02,
  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessi

# RANDOM FOREST

In [30]:
import sklearn.ensemble

model = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=5, random_state=42)
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

In [31]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.7083333333333334

# SVC

In [5]:
import sklearn.svm
import sklearn.model_selection

model = sklearn.svm.SVC(random_state=42)
params = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001]}
gridSearch = sklearn.model_selection.GridSearchCV(model, param_grid=params, cv=5, n_jobs=5, verbose=3)
gridSearch.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


GridSearchCV(cv=5, estimator=SVC(random_state=42), n_jobs=5,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001]},
             verbose=3)

In [7]:
y_hat = gridSearch.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_hat)

0.78125

In [5]:
import pickle
with open('./models/diabetic_retinopathy_svc.pkl', 'rb') as f:
    model = pickle.load(f)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator GridSearchCV from version 1.2.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
from ciu import determine_ciu
import tqdm
import metrics

In [7]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [8]:
feat_list = X_train.columns.tolist()

In [9]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = determine_ciu(X_test.iloc[i:i+1], model.predict_proba, X_train.to_dict('list'), samples = 1000, prediction_index = 1)
        exp_list = [[feat_list.index(i), exp.ci[i]] for i in exp.ci]
        exp1.append(exp_list)
    return np.array(exp1)

In [10]:
exp1 = exp_fn_blk(X_test)
exp2 = exp_fn_blk(X_test)

100%|██████████| 288/288 [02:45<00:00,  1.74it/s]


In [11]:
np.save('./explanations/diabetic_retinopathy_ciu1.npy', exp1)
np.save('./explanations/diabetic_retinopathy_ciu2.npy', exp2)

In [12]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(feat_list))
sb = metrics.calc_stability(enc1, y_test)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [13]:
i, s, sb

((100.0, 0, 288), (0, 288, 82944, 0.0), (100, 288))

In [14]:
X_test_norm = metrics.normalize_test(X_train, X_test)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [15]:
sim

0.9354092777028931

In [16]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in tqdm.tqdm(range(len(X_test))):
    atr = exp1[i]
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(2, dtype=int)
    np.put(y, y_test.iloc[i], 1)
    example = metrics.FeatureAttribution(model, X_test.to_numpy()[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

100%|██████████| 288/288 [00:05<00:00, 57.57it/s]


In [17]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))

-0.053547315878390976
0.013888888888888888
5.461805555555555
-0.08596491228070176
0.0
4.0


In [18]:
metrics.calc_trust_score(model, X_test.to_numpy(), exp1, 3, X_train.columns.to_list())

100%|██████████| 288/288 [00:03<00:00, 80.92it/s]


0.3385416666666667

# RULEFIT

In [4]:
from skrules import SkopeRules
import metrics_rules
import time

In [11]:
# change dataset column names to x0, x1, x2, ...
X_train.columns = ['x'+str(i) for i in range(len(X_train.columns))]
X_test.columns = ['x'+str(i) for i in range(len(X_test.columns))]

In [12]:
clf = SkopeRules(max_depth_duplication=2,
                    n_estimators=512,
                    precision_min=0.3,
                    recall_min=0.1,
                    feature_names=X_train.columns.tolist())

In [13]:
start_time = time.time()
clf.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 20.400280237197876 seconds ---


In [14]:
start_time = time.time()
top_rules1 = clf.score_top_rules(X_test)
top_rules2 = clf.score_top_rules(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.07584309577941895 seconds ---


In [15]:
i = metrics_rules.calc_identity_rules(top_rules1, top_rules2)
print(i)

s = metrics_rules.calc_separability_rules(top_rules1)
print(s)

enc_rules = metrics_rules.exp_enc(clf, top_rules1)
sb = metrics_rules.calc_stability_rules(enc_rules, y_test)
print(sb)

(0.0, 288, 288)
(30466, 288, 82944, 36.730806327160494)
(109, 288)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [16]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test)
sim = metrics_rules.calc_similarity(enc_rules, X_test_norm)
print(sim)

0.8747086026218248
